# Welcome to google image scraping, multi core scrapping directly online



In [2]:
!apt-get install -y chromium-chromedriver
from selenium.webdriver.chrome.service import Service
!apt-get update
!apt-get install -y wget unzip xvfb libxi6 libgconf-2-4
!apt-get install -y google-chrome-stable
!wget https://chromedriver.storage.googleapis.com/134.0.6366.121/chromedriver_linux64.zip
!unzip chromedriver_linux64.zip
!chmod +x chromedriver
!mv chromedriver /usr/local/bin/
!pip install selenium
!pip install webdriver_manager

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [3,962 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,535 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd

In [ ]:
#!/usr/bin/env python3
#Sample queries added
import sys
import csv
import time
import urllib.parse
import os
import concurrent.futures

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import StaleElementReferenceException
from webdriver_manager.chrome import ChromeDriverManager
service = Service(ChromeDriverManager().install())
def parse_imgurl(href):
    """
    Extract the 'imgurl' parameter from a Google '/imgres?' link.
    Returns the high-resolution image URL if found, otherwise None.
    """
    import urllib.parse
    parsed = urllib.parse.urlparse(href)
    qs = urllib.parse.parse_qs(parsed.query)
    return qs.get("imgurl", [None])[0]

def append_to_csv(data_list, csv_filename="images_data.csv"):
    """
    Appends rows to 'csv_filename'. Each row has columns: url, alt.
    If file doesn't exist, it is created, but existing data is preserved.
    """
    import csv
    file_exists = os.path.isfile(csv_filename)
    with open(csv_filename, "a", newline="", encoding="utf-8") as f:
        fieldnames = ["url", "alt"]
        writer = csv.DictWriter(f, fieldnames=fieldnames)

        # If file is newly created, write the header
        if not file_exists:
            writer.writeheader()

        for row in data_list:
            writer.writerow({"url": row["url"], "alt": row["alt"]})

def scrape_google_images(query, max_links=10):
    """
    Runs the Google Images scraping for a single query.
    Returns a list of dicts: [{'url': ..., 'alt': ...}, ...]
    """
    # Setup headless Chrome
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/115.0.0.0 Safari/537.36"
    )

    # If multiple processes try to install or update ChromeDriver simultaneously,
    # you might see "File is not a zip file" or "tuple index out of range".
    # One workaround is to install ChromeDriver once outside the processes,
    # or catch these exceptions.

    driver = webdriver.Chrome( options=chrome_options)

    image_data = []
    seen_urls = set()

    try:
        # 1. Navigate to Google Images
        search_url = f"https://www.google.com/search?tbm=isch&q={urllib.parse.quote(query)}"
        driver.get(search_url)
        time.sleep(2)

        # 2. Scroll to load more containers
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(1)

        # 3. Find containers with class 'ivg-i'
        containers = driver.find_elements(By.CSS_SELECTOR, "div.ivg-i")
        print(f"[{query}] Found {len(containers)} containers.")

        actions = ActionChains(driver)

        for i, container in enumerate(containers):
            if len(image_data) >= max_links:
                break

            try:
                # Hover over the container
                actions.move_to_element(container).perform()
                k=1

                # Find anchor with href^='/imgres?'
                anchor = container.find_element(By.CSS_SELECTOR, "a[href^='/imgres?']")
                href = anchor.get_attribute("href")
                if href and "imgurl=" in href:
                    high_res_url = parse_imgurl(href)
                    if high_res_url and high_res_url not in seen_urls:
                        seen_urls.add(high_res_url)
                        # Attempt to get alt text from child <img>
                        try:
                            child_img = anchor.find_element(By.TAG_NAME, "img")
                            alt_text = child_img.get_attribute("alt") or ""
                        except:
                            alt_text = ""

                        image_data.append({"url": high_res_url, "alt": alt_text})
                        print(f"[{query}] Container #{i+1}: {high_res_url[:80]}... Alt: {alt_text}")
            except StaleElementReferenceException:
                pass
            except Exception as e:
                pass

    finally:
        driver.quit()

    return image_data

def process_query(query, max_links=10, csv_file="images_data.csv"):
    """
    Worker function to scrape images for a single query and append them to a CSV.
    """
    try:
        data = scrape_google_images(query, max_links=max_links)
        append_to_csv(data, csv_filename=csv_file)
        return f"Scraped {len(data)} images for '{query}'."
    except Exception as e:
        # If you get "File is not a zip file" or others, handle them here
        return f"Error scraping '{query}': {e}"

def main():

    queries = [
        "bose headphones",
        "sony headphones",
        "jbl headphones",
    ]
    max_links = 1000
    csv_name = "images.csv"
    # We'll run them in parallel using ProcessPoolExecutor
    import concurrent.futures

    with concurrent.futures.ProcessPoolExecutor() as executor:
        # Submit tasks
        futures = {executor.submit(process_query, q, max_links,csv_name): q for q in queries}

        for future in concurrent.futures.as_completed(futures):
            q = futures[future]
            try:
                result = future.result()
                print(f"[{q}] {result}")
            except Exception as exc:
                print(f"[{q}] generated an exception: {exc}")

if __name__ == "__main__":
    main()


In [68]:
#!/usr/bin/env python3

import sys
import csv
import time
import urllib.parse
import os
import concurrent.futures

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
service = Service(ChromeDriverManager().install())
def parse_imgurl(href):
    """
    Extract the 'imgurl' parameter from a Google '/imgres?' link.
    Returns the high-resolution image URL if found, otherwise None.
    """
    import urllib.parse
    parsed = urllib.parse.urlparse(href)
    qs = urllib.parse.parse_qs(parsed.query)
    return qs.get("imgurl", [None])[0]

def append_to_csv(data_list, csv_filename="images_data.csv"):
    """
    Appends rows to 'csv_filename'. Each row has columns: url, alt.
    If file doesn't exist, it is created, but existing data is preserved.
    """
    import csv
    file_exists = os.path.isfile(csv_filename)
    with open(csv_filename, "a", newline="", encoding="utf-8") as f:
        fieldnames = ["url", "alt"]
        writer = csv.DictWriter(f, fieldnames=fieldnames)

        # If file is newly created, write the header
        if not file_exists:
            writer.writeheader()

        for row in data_list:
            writer.writerow({"url": row["url"], "alt": row["alt"]})

def scrape_google_images(query, max_links=10):
    """
    Runs the Google Images scraping for a single query.
    Returns a list of dicts: [{'url': ..., 'alt': ...}, ...]
    """
    # Setup headless Chrome
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/115.0.0.0 Safari/537.36"
    )

    # If multiple processes try to install or update ChromeDriver simultaneously,
    # you might see "File is not a zip file" or "tuple index out of range".
    # One workaround is to install ChromeDriver once outside the processes,
    # or catch these exceptions.

    driver = webdriver.Chrome( options=chrome_options)

    image_data = []
    seen_urls = set()

    try:
        # 1. Navigate to Google Images
        search_url = f"https://www.google.com/search?tbm=isch&q={urllib.parse.quote(query)}"
        driver.get(search_url)
        time.sleep(2)

        # 2. Scroll to load more containers
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(1)

        # 3. Find containers with class 'ivg-i'
        containers = driver.find_elements(By.CSS_SELECTOR, "div.ivg-i")
        print(f"[{query}] Found {len(containers)} containers.")

        actions = ActionChains(driver)

        for i, container in enumerate(containers):
            if len(image_data) >= max_links:
                break

            try:
                # Hover over the container
                actions.move_to_element(container).perform()
                k=1

                # Find anchor with href^='/imgres?'
                anchor = container.find_element(By.CSS_SELECTOR, "a[href^='/imgres?']")
                href = anchor.get_attribute("href")
                if href and "imgurl=" in href:
                    high_res_url = parse_imgurl(href)
                    if high_res_url and high_res_url not in seen_urls:
                        seen_urls.add(high_res_url)
                        # Attempt to get alt text from child <img>
                        try:
                            child_img = anchor.find_element(By.TAG_NAME, "img")
                            alt_text = child_img.get_attribute("alt") or ""
                        except:
                            alt_text = ""

                        image_data.append({"url": high_res_url, "alt": alt_text})
                        print(f"[{query}] Container #{i+1}: {high_res_url[:80]}... Alt: {alt_text}")
            except StaleElementReferenceException:
                pass
            except Exception as e:
                pass

    finally:
        driver.quit()

    return image_data

def process_query(query, max_links=10, csv_file="images_data.csv"):
    """
    Worker function to scrape images for a single query and append them to a CSV.
    """
    try:
        data = scrape_google_images(query, max_links=max_links)
        append_to_csv(data, csv_filename=csv_file)
        return f"Scraped {len(data)} images for '{query}'."
    except Exception as e:
        # If you get "File is not a zip file" or others, handle them here
        return f"Error scraping '{query}': {e}"

def main():
    if len(sys.argv) < 2:
        print("Usage: python script.py [<csv_name>] <query1> [<query2> ...] [<max_links>]")
        sys.exit(1)

    queries = sys.argv[2:-1]  # All arguments except the last one (if it's a number)
    try:
        max_links = int(sys.argv[-1]) if sys.argv[-1].isdigit() else 100
    except ValueError:
        max_links = 100
    csv_name = sys.argv[1] if len(sys.argv) > 1 else "images_data.csv"


    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = {executor.submit(process_query, q, max_links): q for q in queries}
        for future in concurrent.futures.as_completed(futures):
            q = futures[future]
            try:
                result = future.result()
                print(f"[{q}] {result}")
            except Exception as exc:
                print(f"[{q}] Exception: {exc}")

if __name__ == "__main__":
    main()
